In [18]:
using Base.Threads

In [19]:
Threads.nthreads()

24

In [20]:
pwd()

"/home/xyu/github/BulkLMM.jl/analysis"

In [21]:
cd("..")

In [22]:
include("../test/BXDdata_for_test.jl");

LoadError: SystemError: opening file "/home/xyu/github/test/BXDdata_for_test.jl": No such file or directory

In [4]:
include("../src/parallel_helpers.jl");

In [5]:
include("../test/testHelper.jl");

In [6]:
# using Pkg
# Pkg.add("ThreadTools")
using ThreadTools

In [7]:
?tmap

search: tmap tmap1 @atomicswap BitMatrix Cintmax_t floatmax @fastmath Cuintmax_t



```
tmap(f, args...)
tmap(f, v, T::DataType)
tmap(f, nthreads::Int, args...)
tmap1(f, args...)
tmap1(f, nthreads::Int, args...)
```

Threaded map. The optional argument `nthreads` limits the number of threads used in parallel. `tmap1` is the same as `tmap`, but falls back to a regular `map` if julia only has access to one thread. If the eltype `T` of the output is specified the call will be type stable.


In [8]:
function threads12_by_blocks(r0perm::Array{Float64, 2}, X00::Array{Float64, 2}, nblocks::Int64)
    # Does distributed processes of calculations of LOD scores for markers in each block

    p = size(X00, 2);

    ## (Create blocks...)
    
    block_size = ceil(Int, p/nblocks);
    blocks = createBlocks2(p, block_size);
    # blocks = createBlocks(p, nblocks);

    LODs_blocks = tmap(x -> calcLODs_block(r0perm, X00, x), 24, blocks);
    results = reduce(hcat, LODs_blocks);

    return results

end

threads12_by_blocks (generic function with 1 method)

In [9]:
function scan_perms_threads12(y::Array{Float64,2}, g::Array{Float64,2}, K::Array{Float64,2};
                                reml::Bool = false,
                                nperms::Int64 = 1024, rndseed::Int64 = 0, original::Bool = true,
                                # (options for blocks, nperms distribution methods...)
                                option::String = "by blocks", nblocks::Int64 = 1, ncopies::Int64 = 1, 
                                nprocs::Int64 = 0)

    (y0, X0, lambda0) = transform_rotation(y, g, K); # rotation of data
    (r0, X00) = transform_reweight(y0, X0, lambda0; reml = reml); # reweighting and taking residuals
    r0perm = transform_permute(r0; nperms = nperms, rndseed = rndseed, original = original);

    if option == "by blocks"
        results = threads12_by_blocks(r0perm, X00, nblocks);
    elseif option == "by nperms"
        results = distribute_by_nperms(r0, X00, nperms, ncopies, original);
    else
        throw(error("Option unsupported."))
    end

    return results

end

scan_perms_threads12 (generic function with 1 method)

In [10]:
@benchmark scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 240)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.109 s …    2.646 s  ┊ GC (min … max): 13.05% … 30.60%
 Time  (median):     2.134 s               ┊ GC (median):    18.24%
 Time  (mean ± σ):   2.296 s ± 303.313 ms  ┊ GC (mean ± σ):  21.40% ±  9.02%

  █ █                                                      █  
  █▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.11 s         Histogram: frequency by time         2.65 s <

 Memory estimate: 13.68 GiB, allocs estimate: 79448.

In [11]:
@benchmark scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 120)

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  1.776 s …    2.225 s  ┊ GC (min … max):  6.01% … 28.41%
 Time  (median):     1.781 s               ┊ GC (median):     5.99%
 Time  (mean ± σ):   1.927 s ± 257.947 ms  ┊ GC (mean ± σ):  14.59% ± 12.97%

  █                                                        ▁  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.78 s         Histogram: frequency by time         2.23 s <

 Memory estimate: 13.67 GiB, allocs estimate: 77096.

In [12]:
@benchmark scan_perms(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.924 s …  2.929 s  ┊ GC (min … max): 13.61% … 13.08%
 Time  (median):     2.927 s             ┊ GC (median):    13.35%
 Time  (mean ± σ):   2.927 s ± 3.571 ms  ┊ GC (mean ± σ):  13.35% ±  0.37%

  █                                                      █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.92 s        Histogram: frequency by time        2.93 s <

 Memory estimate: 13.61 GiB, allocs estimate: 74659.

In [17]:
tmap_LODs = scan_perms_threads12(pheno_y, geno, kinship; reml = false, nperms = 1024, rndseed = 0, original = true,
                                             option = "by blocks", nblocks = 320);
tmap_LODs[1:6, :]

6×7321 Matrix{Float64}:
 0.00819636  0.00819636  0.00819636  …  0.0449878  0.0128283  0.0128283
 0.604956    0.604956    0.604956       1.33928    1.29426    1.29426
 0.0264842   0.0264842   0.0264842      0.16927    0.069518   0.069518
 0.00585467  0.00585467  0.00585467     0.589608   0.14854    0.14854
 0.477297    0.477297    0.477297       0.0317133  0.0625366  0.0625366
 0.140293    0.140293    0.140293    …  0.0400303  0.0338204  0.0338204